In [5]:
!nvidia-smi

Wed Dec  9 21:31:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
os.chdir("/content/drive/MyDrive/Tensorflow Advance/Practice/CatDog/data")
import numpy as np
print("version of tensorflow is : ",tf.__version__)

In [6]:
train_dir = "./train/"
test_dir  = "./test/"

In [ ]:
base_model = InceptionV3(include_top=False , weights='imagenet',input_shape=(150 , 150 ,3))
base_model.summary()

In [8]:
needtotrain = False
for layer in base_model.layers:
    if layer.name == 'conv2d_93':
        needtotrain = True
    layer.trainable = needtotrain

In [9]:
print("Shape of output for base model : ",base_model.output_shape)

Shape of output for base model :  (None, 3, 3, 2048)


In [10]:
inp = tf.keras.layers.Input(shape=(150 , 150 , 3))
x = base_model(inp)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64 , activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(128 , activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(1   , activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs = inp , outputs = x)
model.compile(loss=tf.keras.losses.binary_crossentropy , optimizer='rmsprop' , metrics=['acc'])

In [11]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 3, 3, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1179712   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 22,990,945
Trainable params: 1,581,889
Non-trainable params: 21,409,056
__________________________________

In [12]:
train_data_gen = ImageDataGenerator(rescale=(1.0 / 255.0) ,
                                    width_shift_range=0.2 ,
                                    height_shift_range=0.2 ,
                                    zoom_range=0.2 ,
                                    rotation_range=35)

test_data_gen  = ImageDataGenerator(rescale=(1.0 / 255.0))

In [13]:
train_data = train_data_gen.flow_from_directory(train_dir , target_size=(150 , 150) , class_mode='binary')
test_data  = test_data_gen.flow_from_directory(test_dir , target_size=(150 , 150) , class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [15]:
his = model.fit(train_data ,
                epochs=10 ,
                steps_per_epoch=train_data.n // train_data.batch_size ,
                validation_data=test_data ,
                validation_steps=test_data.n // test_data.batch_size)

Epoch 1/10
625/625 [==============================] - 11760s 19s/step - loss: 0.3409 - acc: 0.9048 - val_loss: 0.1266 - val_acc: 0.9477
Epoch 2/10
625/625 [==============================] - 183s 293ms/step - loss: 0.1742 - acc: 0.9351 - val_loss: 0.0926 - val_acc: 0.9710
Epoch 3/10
625/625 [==============================] - 182s 290ms/step - loss: 0.1594 - acc: 0.9406 - val_loss: 0.0775 - val_acc: 0.9756
Epoch 4/10
625/625 [==============================] - 180s 288ms/step - loss: 0.1484 - acc: 0.9426 - val_loss: 0.0884 - val_acc: 0.9708
Epoch 5/10
625/625 [==============================] - 181s 289ms/step - loss: 0.1427 - acc: 0.9460 - val_loss: 0.0783 - val_acc: 0.9726
Epoch 6/10
625/625 [==============================] - 179s 287ms/step - loss: 0.1397 - acc: 0.9477 - val_loss: 0.1015 - val_acc: 0.9746
Epoch 7/10
625/625 [==============================] - 180s 288ms/step - loss: 0.1408 - acc: 0.9487 - val_loss: 0.0810 - val_acc: 0.9734
Epoch 8/10
625/625 [============================

In [16]:
model.save("/content/drive/MyDrive/Tensorflow Advance/Advanced Computer Vision with TensorFlow/Week1/inceptionmodel.h5")